# Model v1
Najprostszy model, tylko suma energi grupowana po czasie i lokalizacji. Dense + LSTM.

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import h5py
from sklearn.preprocessing import MinMaxScaler, StandardScaler

2023-09-17 13:47:11.268142: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-17 13:47:14.679816: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df_full = pd.read_csv("eq/merge.csv")

/tmp/ipykernel_681/2632980415.py:1: DtypeWarning: Columns (7,11,13,14,24,25,26,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv("eq/merge.csv")


In [3]:
df_full.columns

Index(['network_code', 'receiver_code', 'receiver_type', 'receiver_latitude',
       'receiver_longitude', 'receiver_elevation_m', 'p_arrival_sample',
       'p_status', 'p_weight', 'p_travel_sec', 's_arrival_sample', 's_status',
       's_weight', 'source_id', 'source_origin_time',
       'source_origin_uncertainty_sec', 'source_latitude', 'source_longitude',
       'source_error_sec', 'source_gap_deg',
       'source_horizontal_uncertainty_km', 'source_depth_km',
       'source_depth_uncertainty_km', 'source_magnitude',
       'source_magnitude_type', 'source_magnitude_author',
       'source_mechanism_strike_dip_rake', 'source_distance_deg',
       'source_distance_km', 'back_azimuth_deg', 'snr_db', 'coda_end_sample',
       'trace_start_time', 'trace_category', 'trace_name'],
      dtype='object')

In [4]:
df_full = df_full[df_full["trace_category"] == "earthquake_local"]

In [5]:
cols = ['trace_start_time', 'source_latitude', 'source_longitude', 'source_magnitude']

In [67]:
df_main = df_full[cols].dropna().reset_index(drop=True)
df = df_main.copy(deep=True)

In [68]:
# df = df[df["source_magnitude"] < 0]

In [69]:
df["trace_start_time"] = pd.to_datetime(df["trace_start_time"])

In [70]:
df.describe()

,trace_start_time,source_latitude,source_longitude,source_magnitude
count,1030231,1.030231e+06,1.030231e+06,1.030231e+06
mean,2014-06-05 13:50:09.258782720,4.045908e+01,-1.167180e+02,1.526000e+00
min,1984-04-24 22:18:07.960000,-4.384550e+01,-1.799965e+02,-5.000000e-01
25%,2012-07-23 04:49:00.325000192,3.378317e+01,-1.243928e+02,8.000000e-01
50%,2014-11-08 12:14:40.710000128,3.755617e+01,-1.188468e+02,1.300000e+00
75%,2017-01-14 02:36:01.304604928,4.571090e+01,-1.165028e+02,2.000000e+00
max,2018-12-31 23:54:28.830000,7.838040e+01,1.799972e+02,7.900000e+00
std,NaN,1.379810e+01,4.199186e+01,9.757983e-01


In [71]:
df.head()

,trace_start_time,source_latitude,source_longitude,source_magnitude
0,2006-07-23 15:59:00.960,33.7496,-117.4938,3.6
1,2006-11-03 15:56:53.610,32.7077,-116.0446,4.3
2,2006-11-03 16:12:24.700,32.7253,-116.0348,3.6
3,2006-11-14 13:32:22.540,32.7063,-116.0241,3.8
4,2006-11-27 10:46:41.060,31.9679,-117.1944,3.6


In [72]:
df["label"] = np.where(df["source_magnitude"] >= 4, 1, 0)
df["energy"] = np.power(10, 1.44 * df["source_magnitude"] + 5.24)

In [73]:
geo_split = 10
time_split = 30

In [74]:
df["source_latitude"] = df["source_latitude"] // geo_split
df["source_latitude"] = df["source_latitude"] + np.abs(np.min(df["source_latitude"]))

df["source_longitude"] = df["source_longitude"] // geo_split
df["source_longitude"] = df["source_longitude"] + np.abs(np.min(df["source_longitude"]))

In [75]:
df

,trace_start_time,source_latitude,source_longitude,source_magnitude,label,energy
0,2006-07-23 15:59:00.960,8.0,6.0,3.60,0,2.654606e+10
1,2006-11-03 15:56:53.610,8.0,6.0,4.30,1,2.703958e+11
2,2006-11-03 16:12:24.700,8.0,6.0,3.60,0,2.654606e+10
3,2006-11-14 13:32:22.540,8.0,6.0,3.80,0,5.152286e+10
4,2006-11-27 10:46:41.060,8.0,6.0,3.60,0,2.654606e+10
...,...,...,...,...,...,...
1030226,2017-06-21 09:40:43.810,9.0,6.0,0.39,0,6.332862e+05
1030227,2017-06-21 12:21:05.390,9.0,6.0,1.72,0,5.209547e+07
1030228,2017-06-21 12:23:40.200,9.0,6.0,1.61,0,3.617429e+07
1030229,2017-06-21 13:34:40.330,9.0,6.0,1.25,0,1.096478e+07


In [76]:
# group the data into 30 days intervals
df["trace_start_time"] = df["trace_start_time"].dt.floor('d')
df["trace_start_time"] = df["trace_start_time"] - np.min(df["trace_start_time"])
df["trace_start_time"] = df["trace_start_time"].dt.days // time_split
df["trace_start_time"] = df["trace_start_time"].astype(int)


In [77]:
df_agg = df.groupby(["source_latitude", "source_longitude", "trace_start_time"]).sum()
df_agg

source_magnitude  label  \
source_latitude source_longitude trace_start_time                            
0.0             35.0             176                            7.5      0   
                                 186                            4.6      1   
                                 195                            4.4      1   
                                 198                            3.7      0   
                                 199                            4.3      1   
...                                                             ...    ...   
12.0            11.0             370                            4.4      1   
                12.0             364                            4.2      0   
                                 370                            2.5      0   
                                 371                            2.3      0   
                18.0             266                            2.5      0   

                                                         energy  
source_latitude source_longitude trace_start_time                
0.0             35.0             176               9.832548e+10  
                                 186               7.311391e+11  
                                 195               3.767038e+11  
                                 198               3.698282e+10  
                                 199               2.703958e+11  
...                                                         ...  
12.0            11.0             370               3.767038e+11  
                12.0             364               3.673077e+08  
                                 370               6.918310e+08  
                                 371               3.564511e+08  
                18.0             266               6.918310e+08  

[6560 rows x 3 columns]

In [78]:
df_agg = df_agg.reset_index()

In [79]:
scaler = MinMaxScaler()
df_agg["label"] = np.where(df_agg["label"] >= 1, 1, 0)
df_agg["energy"] = scaler.fit_transform(df_agg["energy"].values.reshape(-1, 1))

In [80]:
df_agg["label"].value_counts()

label
0    3995
1    2565
Name: count, dtype: int64

In [81]:
block_size = 16
dfs = []
for i in df_agg["source_latitude"].unique():
    for j in df_agg["source_longitude"].unique():
        tmp = df_agg[(df_agg["source_latitude"] == i) & (df_agg["source_longitude"] == j)]
        # add the missing time intervals, fill with 0
        if not tmp.empty:
            tmp = tmp.set_index("trace_start_time").reindex(range(tmp["trace_start_time"].min() - block_size, df_agg["trace_start_time"].max()+1)).fillna(0).rename_axis('trace_start_time').reset_index()
            tmp["label"] = tmp["label"].shift(-1)
            for idx in  range(1, block_size):
                tmp["energy" + str(idx)] = tmp["energy"].shift(idx)
            tmp["source_latitude"] = i
            tmp["source_longitude"] = j
            dfs.append(tmp)
tmp = pd.concat(dfs)
tmp


,trace_start_time,source_latitude,source_longitude,source_magnitude,label,energy,energy1,energy2,energy3,energy4,...,energy6,energy7,energy8,energy9,energy10,energy11,energy12,energy13,energy14,energy15
0,160,0.0,35.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,161,0.0,35.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,162,0.0,35.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,163,0.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,164,0.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,418,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71,419,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,420,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73,421,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [82]:
tmp.dropna(inplace=True)

In [83]:
tmp

,trace_start_time,source_latitude,source_longitude,source_magnitude,label,energy,energy1,energy2,energy3,energy4,...,energy6,energy7,energy8,energy9,energy10,energy11,energy12,energy13,energy14,energy15
15,175,0.0,35.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,176,0.0,35.0,7.5,0.0,0.000001,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,177,0.0,35.0,0.0,0.0,0.000000,0.000001,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,178,0.0,35.0,0.0,0.0,0.000000,0.000000,0.000001,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19,179,0.0,35.0,0.0,0.0,0.000000,0.000000,0.000000,0.000001,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,417,12.0,12.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70,418,12.0,12.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71,419,12.0,12.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,420,12.0,12.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
tmp["label"].value_counts()

label
0.0    18614
1.0     2565
Name: count, dtype: int64

In [85]:
x = tmp.drop(["trace_start_time", "source_latitude", "source_longitude", "source_magnitude","label"], axis=1).to_numpy()
y = tmp["label"].to_numpy()

In [86]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(block_size, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2)
])
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [87]:
model.fit(x, y, epochs=10, batch_size=128, validation_split=0.2, shuffle=False)

Epoch 1/10


133/133 [==============================] - 4s 12ms/step - loss: 0.5259 - accuracy: 0.8706 - val_loss: 0.3560 - val_accuracy: 0.8872
Epoch 2/10
133/133 [==============================] - 2s 14ms/step - loss: 0.3754 - accuracy: 0.8768 - val_loss: 0.3516 - val_accuracy: 0.8872
Epoch 3/10
133/133 [==============================] - 2s 12ms/step - loss: 0.3765 - accuracy: 0.8768 - val_loss: 0.3507 - val_accuracy: 0.8874
Epoch 4/10
133/133 [==============================] - 2s 11ms/step - loss: 0.3765 - accuracy: 0.8768 - val_loss: 0.3501 - val_accuracy: 0.8879
Epoch 5/10
133/133 [==============================] - 1s 11ms/step - loss: 0.3765 - accuracy: 0.8768 - val_loss: 0.3496 - val_accuracy: 0.8874
Epoch 6/10
133/133 [==============================] - 2s 11ms/step - loss: 0.3758 - accuracy: 0.8768 - val_loss: 0.3496 - val_accuracy: 0.8879
Epoch 7/10
133/133 [==============================] - 1s 10ms/step - loss: 0.3760 - accuracy: 0.8768 - val_loss: 0.3494 - val_accuracy: 0.8881
Epoch 8/10

In [88]:
np.unique(y, return_counts=True)

(array([0., 1.]), array([18614,  2565]))

In [89]:
x = x.reshape(-1, 1, block_size)
x.shape

(21179, 1, 16)

In [90]:
model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)
])
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [91]:
model.fit(x, y, epochs=10, batch_size=512, validation_split=0.2, shuffle=False)

Epoch 1/10


34/34 [==============================] - 13s 87ms/step - loss: 0.6417 - accuracy: 0.8764 - val_loss: 0.5389 - val_accuracy: 0.8872
Epoch 2/10
34/34 [==============================] - 1s 35ms/step - loss: 0.4228 - accuracy: 0.8768 - val_loss: 0.3534 - val_accuracy: 0.8872
Epoch 3/10
34/34 [==============================] - 1s 35ms/step - loss: 0.3766 - accuracy: 0.8768 - val_loss: 0.3506 - val_accuracy: 0.8872
Epoch 4/10
34/34 [==============================] - 1s 35ms/step - loss: 0.3741 - accuracy: 0.8768 - val_loss: 0.3499 - val_accuracy: 0.8872
Epoch 5/10
34/34 [==============================] - 1s 35ms/step - loss: 0.3740 - accuracy: 0.8768 - val_loss: 0.3495 - val_accuracy: 0.8872
Epoch 6/10
34/34 [==============================] - 1s 36ms/step - loss: 0.3739 - accuracy: 0.8768 - val_loss: 0.3491 - val_accuracy: 0.8872
Epoch 7/10
34/34 [==============================] - 1s 36ms/step - loss: 0.3737 - accuracy: 0.8768 - val_loss: 0.3488 - val_accuracy: 0.8872
Epoch 8/10
34/34 [=====

In [60]:
preds = model.predict(x, batch_size=512)
preds = np.argmax(preds, axis=1)
(preds == 0).sum(), (preds == 1).sum()

42/42 [==============================] - 2s 12ms/step


(21178, 1)

In [61]:
(y == 1).sum(), (y == 0).sum()

(502, 20677)

In [92]:
x

array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[1.18125182e-06, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[0.00000000e+00, 1.18125182e-06, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       ...,

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]])